In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import glob
import csv
import pymysql
from ipynb.fs.full.Db_Connection import open_conn
pd.set_option('display.max_column', None)

file_dir = 'D:/Abbas/Extract/LCM/'
lata_info = pd.read_csv(file_dir+'Lata_No.csv', dtype='str')
lata_select = lata_info['LATA_NUMBER'].iloc[0]
lata_list = lata_select.split("-")

summary_report = pd.read_excel('D:/Abbas/Reports/Main_Summary.xlsx', dtype=str)
summary_report.columns = summary_report.columns.str.strip()
summary_report.dropna(how='all', inplace=True)
summary_report = summary_report.replace(np.nan,'')
lata_input = 'D:/Abbas/Extract/LCM/Input/'
lata_output = 'D:/Abbas/Extract/LCM/Output/'

for l in range(len(lata_list)):
    conn = open_conn()
    database = pd.read_sql("""SELECT circuittype as TYPE, NAME as CIRCUIT, lata_a as LATA_A, lata_z as LATA_Z from circuitinv 
    WHERE lata_a = '"""+lata_list[l]+"""' OR lata_z = '"""+lata_list[l]+"'", conn);
    
    allFiles = glob.glob(lata_input+'*'+lata_list[l]+'*')
    l1 = '_LCM'
    lcm_filename = [i for i in allFiles if l1 in i]
    lcm_filename = lcm_filename[0]
    lcm = pd.read_excel(lcm_filename)

    df1 = database
    df2 = lcm
    
    # split lata attached to circuits
    df1['CIRCUIT'] = df1['CIRCUIT'].str.split('_', expand = True)

    writer = pd.ExcelWriter(lata_output+'L'+lata_list[l]+'_LCM_Maestro.xlsx', engine='xlsxwriter')
    summary_writer = pd.ExcelWriter('D:/Abbas/Extract/LCM/Output/Summary.xlsx', engine='xlsxwriter')
    
    if 'Deleted Ckts' in df2.columns:
        df2 = df2.drop(['Deleted Ckts'], axis=1)

    #Intra LATA
    temp1 = pd.DataFrame()
    temp2 = pd.DataFrame()
    temp1['LATA_A'] = df1['LATA_A'].str.extract(r'([a-zA-Z- ]+)', expand=False).str.strip()
    temp2['LATA_Z'] = df1['LATA_Z'].str.extract(r'([a-zA-Z- ]+)', expand=False).str.strip()
    temp1.dropna(inplace=True)
    temp2.dropna(inplace=True)
    temp1.reset_index(inplace=True)
    temp2.reset_index(inplace=True)
    for i in range(len(temp1)):
        df1['LATA_A'].iloc[temp1['index'].iloc[i]] = lata_list[l]
    for i in range(len(temp2)):
        df1['LATA_Z'].iloc[temp2['index'].iloc[i]] = lata_list[l]
    df_lata = df1.loc[df1['LATA_A'] == lata_list[l]]
    df_lata = df_lata[df_lata['LATA_Z'] == lata_list[l]]
    df_lata = df_lata[df_lata['TYPE'] == 'S']
    df_lata = df_lata.reset_index()
    df_lata.drop(['index'], axis=1, inplace=True)

    # lcm_cir_count = df2['# circuits.1'].iloc[4]
    temp_df_intra = df2[df2['Unnamed: 2'] == 'INTRA']
    lcm_cir_count_intra = temp_df_intra['# circuits.1'].iloc[0]
    maes_cir_count_intra = df_lata.shape[0]

    circuit_count = pd.DataFrame(columns=['CIRCUIT FILE','COUNT'])
    circuit_count = circuit_count.append({'CIRCUIT FILE': 'LCM FILE', 'COUNT':lcm_cir_count_intra}, ignore_index=True)
    circuit_count = circuit_count.append({'CIRCUIT FILE': 'MAESTRO FILE', 'COUNT':maes_cir_count_intra}, ignore_index=True)

    # COnvert Column to List
    df_lcm = df_lata['CIRCUIT']
    df_maestr = df2['A & Z are from L'+lata_list[l]]

    # Inter LATA
    df1_s = df1.loc[df1['TYPE'] == 'S']
    df_lata_a = df1_s.loc[df1_s['LATA_A'] == lata_list[l]]
    df_lata_a = df_lata_a.loc[df_lata_a['LATA_Z'] != lata_list[l]]
    df_lata_z = df1_s.loc[df1_s['LATA_Z'] == lata_list[l]]
    df_lata_z = df_lata_z.loc[df_lata_z['LATA_A'] != lata_list[l]]
    df_lata_a = df_lata_a.reset_index()
    df_lata_z = df_lata_z.reset_index()
    df_lata_a.drop(['index'], axis=1, inplace=True)
    df_lata_z.drop(['index'], axis=1, inplace=True)

    df1_a_z_com = pd.concat([df_lata_a['CIRCUIT'], df_lata_z['CIRCUIT']])
    df1_a_z_com = df1_a_z_com.drop_duplicates()
    a_z_lcm = pd.DataFrame(df1_a_z_com, columns=['CIRCUIT'])
    a_z_lcm = a_z_lcm.dropna(axis=0, subset=['CIRCUIT'])
    a_z_lcm_split = a_z_lcm['CIRCUIT'].str.split('_',n = 1, expand = True)
    if a_z_lcm_split.shape[1] == 2:
        a_z_lcm_split.loc[a_z_lcm_split[1].isin([None]), 1] = 'N/A'
        a_z_lcm_split = a_z_lcm_split[(a_z_lcm_split[1]=='lata224') | (a_z_lcm_split[1]=='N/A')]
        a_z_lcm = pd.DataFrame(a_z_lcm_split[0])
        a_z_lcm = a_z_lcm.rename(columns={0:'CIRCUIT'})
    else:
        a_z_lcm = pd.DataFrame(a_z_lcm, columns=['CIRCUIT'])

    df2_a_z_com = pd.concat([df2['A Originated in L'+lata_list[l]], df2['Z Terminated in L'+lata_list[l]]])
    df2_a_z_com = df2_a_z_com.drop_duplicates()
    a_z_mae = pd.DataFrame(df2_a_z_com, columns=['CIRCUIT'])
    a_z_mae = a_z_mae.dropna(axis=0, subset=['CIRCUIT'])

    temp_df_inter = df2[df2['Unnamed: 2'] == 'INTER']
    lcm_cir_count_inter = temp_df_inter['# circuits.1'].iloc[0]
    maes_cir_count_inter = a_z_lcm.shape[0]
    inter_circuit_count = pd.DataFrame(columns=['CIRCUIT FILE','COUNT'])
    inter_circuit_count = inter_circuit_count.append({'CIRCUIT FILE': 'LCM FILE', 'COUNT':lcm_cir_count_inter}, ignore_index=True)
    inter_circuit_count = inter_circuit_count.append({'CIRCUIT FILE': 'MAESTRO FILE', 'COUNT':maes_cir_count_inter}, ignore_index=True)

    a_z_lcm_list = a_z_lcm['CIRCUIT']
    a_z_mae_list = a_z_mae['CIRCUIT']

    # Not in LCM File
    df_other_lcm_cir_col = pd.DataFrame(columns=['CIRCUIT','COLUMN NAME'])
    b=[]
    q=[]
    w=[]
    e=[]
    # Circuit Comparison from Maestro to LCM
    # Circuits not present in LCM selected columns
    not_in_lcm = df_lata[~df_lata['CIRCUIT'].isin(df_maestr)] 
    not_in_lcm = not_in_lcm.replace('nan', np.nan)
    b = not_in_lcm.dropna(axis=0, subset=['CIRCUIT'])
    
    for i in range(len(b)):
        q = df2.columns[df2.isin([b.iloc[i]['CIRCUIT']]).any()] 
        w.append(q)
        e.append(b.iloc[i]['CIRCUIT'])
    w = pd.DataFrame(w)
    e = pd.DataFrame(e, columns=['CIRCUIT'])
    w['COLUMN NAME'] = w.apply(lambda x: '/'.join(x.dropna().values.tolist()), axis=1)
    w = w.reset_index()
    e = e.reset_index()
    df_other_lcm_cir_col = e.merge(w, on=['index'])
    df_other_lcm_cir_col = df_other_lcm_cir_col[['CIRCUIT', 'COLUMN NAME']].copy()

    # Inter LATA_A Comparison
    df_other_lcm_inter = pd.DataFrame()
    not_in_lcm_inter = a_z_lcm[~a_z_lcm['CIRCUIT'].isin(a_z_mae_list)]
    not_in_lcm_inter = not_in_lcm_inter.reset_index()
    not_in_lcm_inter.drop(['index'], axis=1, inplace=True)
    o = []
    y = []
    u = []
    f = []
    df2_select_col = df2.iloc[:,5:]
    for i in range(len(not_in_lcm_inter)):
        p = df2_select_col.columns[df2_select_col.isin([not_in_lcm_inter.iloc[i]['CIRCUIT']]).any()] 
        o.append(p)
        y.append(not_in_lcm_inter.iloc[i]['CIRCUIT'])
    o = pd.DataFrame(o)
    y = pd.DataFrame(y, columns=['CIRCUIT'])
    o['COLUMN NAME'] = o.apply(lambda x: '/'.join(x.dropna().values.tolist()), axis=1)
    o = o.reset_index()
    y = y.reset_index()
    df_other_lcm_inter = y.merge(o, on=['index'])
    df_other_lcm_inter = df_other_lcm_inter[['CIRCUIT', 'COLUMN NAME']].copy()

    # Not in Maestro Database
    maestro_other_lata = pd.DataFrame(columns=['CIRCUIT', 'LATA_A', 'LATA_Z'])
    circuit = []
    a=[] 
    x=[] 
    y=[] 
    z=[] 
    h=[] 
    # Circuit Comparison from LCM to Maestro 
    # Circuits not presend in Maestro LATA selected columns
    not_in_maestr = df2[~df2['A & Z are from L'+lata_list[l]].isin(df_lcm)]
    not_in_maestr = not_in_maestr.replace('nan', np.nan)
    a = not_in_maestr.dropna(axis=0, subset=['A & Z are from L'+lata_list[l]])
    h = df1[df1['CIRCUIT'].isin(a)]
    if h['CIRCUIT'].empty:
        for i in range(len(a)):
            x = a.iloc[i]['A & Z are from L'+lata_list[l]]
            y = ''
            z = ''
            maestro_other_lata = maestro_other_lata.append({'CIRCUIT': x,
                                                            'LATA_A': y,
                                                           'LATA_Z': z}, ignore_index=True)
    else:
        a = a['A & Z are from L'+lata_list[l]]
        for i in range(len(h)):
            x = h.iloc[i]['CIRCUIT']
            y = h.iloc[i]['LATA_A']
            z = h.iloc[i]['LATA_Z']
            maestro_other_lata = maestro_other_lata.append({'CIRCUIT': x,
                                                            'LATA_A': y,
                                                           'LATA_Z': z}, ignore_index=True)

    # Inter LATA Comparison
    not_in_maestr_inter = a_z_mae[~a_z_mae['CIRCUIT'].isin(a_z_lcm_list)]
    not_in_maestr_inter = not_in_maestr_inter.reset_index()
    not_in_maestr_inter.drop(['index'], axis=1, inplace=True)
    
    
###############################################################################################
    # Summary Report
    if lcm_cir_count_intra == maes_cir_count_intra:
        summary_report['L'+lata_list[l]][34] = 'P'
    else:
        summary_report['L'+lata_list[l]][34] = 'F'
        
    if lcm_cir_count_inter == maes_cir_count_inter:
        summary_report['L'+lata_list[l]][35] = 'P'
    else:
        summary_report['L'+lata_list[l]][35] = 'F'
    
###############################################################################################

    # Save Files
    circuit_count.to_excel(writer, sheet_name='Intra Circuit Count', index=False)
    lcm.to_excel(writer, sheet_name='LCM', index=False)
    df1.to_excel(writer, sheet_name='Maestro', index=False)
    if df_other_lcm_cir_col.empty == False:
        df_other_lcm_cir_col.to_excel(writer, sheet_name='Intra Not_LCM', index=False)
    if maestro_other_lata.empty == False:
        maestro_other_lata.to_excel(writer, sheet_name='Intra Not_Mae', index=False)

    if inter_circuit_count.empty == False:
        inter_circuit_count.to_excel(writer, sheet_name='Inter Circuit Count', index=False)
    if df_other_lcm_inter.empty == False:
        df_other_lcm_inter.to_excel(writer, sheet_name='Inter Not_LCM', index=False)
    if not_in_maestr_inter.empty == False:
        not_in_maestr_inter.to_excel(writer, sheet_name='Inter Not_Mae', index=False)
        
    summary_report.to_excel(summary_writer, sheet_name='Summary', index=False)
    
    writer.save()

summary_writer.save()